### Imports

In [ ]:
import chipwhisperer as cw
import matplotlib.pyplot as plt
import numpy as np
import time
import struct
import random

from scipy.signal import find_peaks

In [ ]:
project_folder = "cw_projects/"
project_name = "new_inputs_protected"
num_traces = 10000
scmd_value = 1 #0 for unprotected and 1 for protected

In [ ]:
# project_name = "unprotected"
# num_traces = 2000
# scmd_value = 0 #0 for unprotected and 1 for protected

In [ ]:
min_in_val = -2
max_in_val = 2
decimate_value = 1
filename = project_name + "-trace.txt"

### Function Definitions

In [ ]:
def random_float(min_val, max_val):
    # Generate a random float between min_val and max_val
    rand_float = random.uniform(min_val, max_val)
    # Round to 2 decimal places
    return round(rand_float, 2)

In [ ]:
def float_to_bytearray_32bit_little_edian(f):
    # Pack the float as a 32-bit (4-byte) IEEE 754 floating point number
    packed = struct.pack('f', f)
    # Convert to bytearray
    return bytearray(packed)

In [ ]:
def scope_setup(samples=24431, decimate=2):
    # arm the scope
    scope.arm()
    
    # Set the maximum number of points in a trace
    scope.adc.fifo_fill_mode = "normal"
    scope.adc.samples = samples
    scope.adc.decimate = decimate

In [ ]:
def capture_trace(cmd_data, cmd='p', scmd=scmd_value, prints=True):
    scope.arm()
    # flush the UART buffer
    target.flush()
    
    target.send_cmd(cmd, scmd, cmd_data)
    ret = scope.capture()
    trace = scope.get_last_trace()
    
    returned_data = target.read_cmd('r')
    ack = target.read_cmd('e')
    if prints:
        print(f'r\t- target.read_cmd("r"):\t{returned_data}')
        print(f'ack\t- target.read_cmd("e"):\t{ack}')
    return trace
    

### Target Setup

In [ ]:
#Scope setup
scope = cw.scope()
scope.default_setup()

target = cw.target(scope, cw.targets.SimpleSerial2) #cw.targets.SimpleSerial can be omitted
#MY CHANGES - changed target to SimpleSerial2 - to be able to send_cmd

In [ ]:
scope_setup(samples=24430, decimate=decimate_value)

In [ ]:
%%bash
cd version_02/
make PLATFORM='CWLITEARM' CRYPTO_TARGET=NONE

In [ ]:
cw.program_target(scope, cw.programmers.STM32FProgrammer, "version_02/simpleserial-target-CWLITEARM.hex")

### Initialize the project

The Chipwhisperer `Project` class can be used to keep a collection of traces. 

In [ ]:
proj = cw.create_project(project_folder + project_name)

In [ ]:
input_vals = []

for i in range(num_traces):
    input_vals.append(random.uniform(min_in_val, max_in_val))

print(max(input_vals))
print(min(input_vals))
print(len(input_vals))

### Trace collection

In [ ]:
start = time.time()
completed_counter = 0

# 50 dummy executions
float_val = -0.657
float_bytearray = float_to_bytearray_32bit_little_edian(float_val)
data = bytearray([0x42] * 4)
for i in range(50):
    trace_wave = capture_trace(float_bytearray, scmd=scmd_value)
print("warm up done")

# real executions
start = time.time()
completed_counter = 0
for i in range(num_traces):
    cmd_data = float_to_bytearray_32bit_little_edian(input_vals[i])
    
    trace_wave = capture_trace(cmd_data=cmd_data, scmd=scmd_value, prints=False)
    trace = cw.Trace(wave=trace_wave, textin=input_vals[i], textout=None, key=None)
    proj.traces.append(trace)
    
    completed_counter += 1
    if completed_counter % 100 == 0: 
        print(f'completed {completed_counter} traces in\t{time.time() - start} seconds' )

end = time.time()
print(f'capturing traces finished in {end - start} seconds!')

In [ ]:
# proj.save()
proj.close()

### Plot trace

In [ ]:
proj = cw.open_project(project_folder + project_name)

In [ ]:
print(len(proj.traces))

In [ ]:
trace_waves_arr = []
for trace in proj.traces:
    trace_waves_arr.append(trace.wave)

In [ ]:
def disconnect_DUT():
    scope.dis()
    target.dis()
    return
disconnect_DUT()

### Save traces as txt files

In [ ]:
proj = cw.open_project(project_folder + project_name)

In [ ]:
trace_waves_arr = []
inputs_arr = []
for trace in proj.traces:
    trace_waves_arr.append(trace.wave)
    inputs_arr.append(trace.textin)

trace_waves_arr = np.array(trace_waves_arr)
print(len(trace_waves_arr))
print(len(inputs_arr))

In [ ]:
import os

def save_files(folder, array, input_file, input_array):
    isExist = os.path.exists(folder)
    if not isExist:
        os.makedirs(folder)
    no_of_traces = len(input_array)
    for n in range(no_of_traces):
        with open(folder + "/trace_"+str(n)+".txt","w+") as file:
            for record in array[n]:
                file.write(str(record)+"\n")
        if n % 100 == 0:
            print(f'Converted {n} traces')
        file.close()
    
    with open(folder + "/" + input_file,"w+") as file:
        for i in range(no_of_traces):
            file.write(str(input_array[i])+"\n")
        file.close()
    return

In [ ]:
print(len(trace_waves_arr[0]))

In [ ]:
save_files("txt_traces/" + project_name, trace_waves_arr, "inputs.txt", inputs_arr)

### Zip files

In [ ]:
import shutil
shutil.make_archive(project_name, 'zip', project_name)
# shutil.make_archive(output_filename_dont_add_.zip, 'zip', directory_to_download)